## kaggle上的一个例子

我想通过这个例子学习学习如何分析数据和分类的流程。

In [1]:
# 首先引入需要用的包

import pandas as pd # pandas专门用来进行数据分析，蛮有用的
import numpy as np
from pandas import Series,DataFrame

# 加载数据，看看数据长什么样
data_train = pd.read_csv('Train.csv')
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


可以看到有10多个字段：
>- PassengerId:乘客ID
>- Survived:是否获救
>- Pclass:乘客等级
>- Name:乘客姓名
>- Sex:性别
>- Age:年龄
>- SibSp:堂兄弟妹个数
>- Parch:父母与小孩个数
>- Ticket:船票信息
>- Fare:票价
>- Cabin:客舱
>- Embarked:登船港口

In [2]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


先看看数据的基本信息，RangeIndex说总共有891个数据，列数有12列；并且能够看到每列数据的类型，而且有些数据是不全的，比如Age只有714个数，Cabin只有204个数。

In [3]:
data_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


以上函数更能描述数据的一些事情。我们可以看到，上面的表格中将特征中数值型数据的一些属性计算了出来，可以一目了然的看到每类数值型特征的平均值，标准差，最小值等属性。比如Age一栏，看到乘客的平均年龄为29.6岁。

接下来继续分析数据。看完了每个数据单独的关系，现在应该结合我们最终的目的来看看数据间相互的关系了。我们最终是分类，也就是想找出哪些特征与最终是否获救有关。因此我们可以看看每个或多个属性和最后获救Survived数据有什么样的关系。

In [4]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set(alpha=0.2) # 设定图表颜色alpha参数

# 在一张大图里分列几个小图
plt.subplot2grid((2,3),(0,0))
data_train.Survived.value_counts().plot(kind='bar')
plt.title(u"获救情况(1为获救)")
plt.ylabel(u"人数")

plt.subplot2grid((2,3),(0,1))
data_train.Pclass.value_counts().plot(kind='bar')
plt.ylabel(u"人数")
plt.title(u"乘客等级分布")

plt.subplot2grid((2,3),(0,2))
plt.scatter(data_train.Survived,data_train.Age)
plt.ylabel(u"年龄")
plt.grid(b=True,which='major',axis='y')
plt.title(u"按年龄看获救分布(1为获救)")

plt.subplot2grid((2,3),(1,0),colspan=2)
data_train.Age[data_train.Pclass == 1].plot(kind='kde')
data_train.Age[data_train.Pclass == 2].plot(kind='kde')
data_train.Age[data_train.Pclass == 3].plot(kind='kde')
plt.xlabel(u"年龄")
plt.ylabel(u"密度")
plt.title(u"各等级的乘客年龄分布")
plt.legend((u'头等舱',u'2等舱',u'3等舱'),loc='best')

plt.subplot2grid((2,3),(1,2))
data_train.Embarked.value_counts().plot(kind='bar')
plt.title(u"各登船口岸人数")
plt.ylabel(u"人数")

plt.show()

/Users/wangsenke/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


由上面的图可以看到，获救情况不容乐观，300多点，一般都没到；3等舱的人蛮多的；从年龄来看，遇难和获救的人跨度都很广；

In [5]:
# 再来看看各乘客等级的获救情况
fig = plt.figure()
fig.set(alpha=0.2)

Survived_0 = data_train.Pclass[data_train.Survived == 0].value_counts()
Survived_1 = data_train.Pclass[data_train.Survived == 1].value_counts()
df = pd.DataFrame({u'获救':Survived_1,u'未获救':Survived_0})
df.plot(kind='bar',stacked=True)
plt.title(u"各乘客等级的获救情况")
plt.xlabel(u"乘客等级")
plt.ylabel(u"人数")

plt.show()

可以看到在特等舱里面获救的几率就要大得多呢？果然有钱有权的就是好。

In [6]:
# 接着看港口的位置是否有影响
fig = plt.figure()
fig.set(alpha=0.2)

Survived_E0 = data_train.Embarked[data_train.Survived == 0].value_counts()
Survived_E1 = data_train.Embarked[data_train.Survived == 1].value_counts()
df = pd.DataFrame({u"获救":Survived_E1,u"未获救":Survived_E0})
df.plot(kind='bar',stacked=True)
plt.title(u"各登陆港口乘客的获救情况")
plt.xlabel(u"登陆港口")
plt.ylabel(u"人数")

plt.show()

In [7]:
# 再看各性别的获救情况
fig = plt.figure()
fig.set(alpha=0.2)

Survived_sex0 = data_train.Sex[data_train.Survived == 0].value_counts()
Survived_sex1 = data_train.Sex[data_train.Survived == 1].value_counts()
df = pd.DataFrame({u'获救':Survived_sex1,u'未获救':Survived_sex0})
df.plot(kind='bar',stacked=True)
plt.title(u"按性别看获救情况")
plt.xlabel(u"性别")
plt.ylabel(u"人数")
plt.show()


这幅图上可以看出，船上的人确实都蛮绅士的，贯彻落实副船长所说的”让小孩和女士先走“的政策。

In [8]:
# 看看各种等级舱里面的性别情况呢？

fig=plt.figure()
fig.set(alpha=0.65) # 设置图像透明度，无所谓
plt.title(u"根据舱等级和性别的获救情况")

ax1 = fig.add_subplot(141)
data_train.Survived[data_train.Sex == 'female'][data_train.Pclass != 3].value_counts().plot(kind='bar',label='female highclass',color='#FA2479')
ax1.set_xticklabels([u"获救",u"未获救"],rotation=0)
ax1.legend([u"女性/高级舱"], loc='best')

ax2=fig.add_subplot(142, sharey=ax1)
data_train.Survived[data_train.Sex == 'female'][data_train.Pclass == 3].value_counts().plot(kind='bar', label='female, low class', color='pink')
ax2.set_xticklabels([u"未获救", u"获救"], rotation=0)
plt.legend([u"女性/低级舱"], loc='best')

ax3=fig.add_subplot(143, sharey=ax1)
data_train.Survived[data_train.Sex == 'male'][data_train.Pclass != 3].value_counts().plot(kind='bar', label='male, high class',color='lightblue')
ax3.set_xticklabels([u"未获救", u"获救"], rotation=0)
plt.legend([u"男性/高级舱"], loc='best')

ax4=fig.add_subplot(144, sharey=ax1)
data_train.Survived[data_train.Sex == 'male'][data_train.Pclass == 3].value_counts().plot(kind='bar', label='male low class', color='steelblue')
ax4.set_xticklabels([u"未获救", u"获救"], rotation=0)
plt.legend([u"男性/低级舱"], loc='best')

plt.show()

In [9]:
# 再看看其他的获救情况
fig = plt.figure()
fig.set(alpha=0.2)

Survived_S0 = data_train.SibSp[data_train.Survived == 0].value_counts()
Survived_S1 = data_train.SibSp[data_train.Survived == 1].value_counts()
df = pd.DataFrame({u'获救':Survived_S1,u'未获救':Survived_S0})
df.plot(kind='bar',stacked=True)
plt.title(u"按堂兄妹看获救情况")
plt.xlabel(u"堂兄妹个数")
plt.ylabel(u"人数")
plt.show()

In [10]:
g = data_train.groupby(['Parch','Survived'])
df = pd.DataFrame(g.count()['PassengerId'])
df

PassengerId
Parch Survived             
0     0                 445
      1                 233
1     0                  53
      1                  65
2     0                  40
      1                  40
3     0                   2
      1                   3
4     0                   4
5     0                   4
      1                   1
6     0                   1

好吧，数据里有些比如船票应该是1人1张的，和最后是否获救没啥关系，因此不用看了。还有就是有些数据不全的值，比如说Cabin，只有204个数据，这也是我们需要分析分析的,先来看看它的分布。

In [11]:
# cabin
data_train.Cabin.value_counts()

C23 C25 C27        4
G6                 4
B96 B98            4
D                  3
C22 C26            3
E101               3
F2                 3
F33                3
B57 B59 B63 B66    2
C68                2
B58 B60            2
E121               2
D20                2
E8                 2
E44                2
B77                2
C65                2
D26                2
E24                2
E25                2
B20                2
C93                2
D33                2
E67                2
D35                2
D36                2
C52                2
F4                 2
C125               2
C124               2
                  ..
F G63              1
A6                 1
D45                1
D6                 1
D56                1
C101               1
C54                1
D28                1
D37                1
B102               1
D30                1
E17                1
E58                1
F E69              1
D10 D12            1
E50                1
A14          

这Cabin数据也是。。一眼看上去感觉很多又乱，每个就那么几个。。一个一个看也太麻烦了。。就先看看Cabin这个数据有或没有两种情况吧。

In [12]:
# Cabin值太分散了，很多都是只出现了1次，因此先看看这个值的有无，对于survival的分布状况
fig = plt.figure()
fig.set(alpha=0.2)

Survived_cabin = data_train.Survived[pd.notnull(data_train.Cabin)].value_counts()
Survived_nocabin = data_train.Survived[pd.isnull(data_train.Cabin)].value_counts()
df = pd.DataFrame({u"登记了船舱":Survived_cabin,u"没登记船舱":Survived_nocabin}).transpose()
df.plot(kind='bar',stacked=True)
plt.title(u"按Cabin有无看获救情况")
plt.xlabel(u"Cabin有无")
plt.ylabel(u"人数")

plt.show()

看样子好像有船舱记录的获救比例高些啊，那么就这样把Cabin分成Yes和No吧，作为一个特征。

接下来还有一个拥有缺失值的属性Age年龄，其实这个属性应该还是有些关键的，因为副船长说过嘛，小孩也是优先级高的，因此Age或许是个比较大的影响因子。但是作为候选特征的Age数据却不全，这怎么办呢？

遇到缺值的情况，一般有几种常见的处理方式：
>1. 如果缺失值的样本占总数的比例极高，我们可能直接丢弃，因为它本来就没几个嘛，或许是噪点呢。
>2. 如果缺失值的样本适中，而且该属性不是连续值的特征（就像刚才的Cabin，可以看做类目属性），就可以把NaN作为一个新类别，加入到类别特征中去；（刚才是因为Cabin里类目太分散了，每个的数量又很稀少，因此就定了有和无两个类目）
>3. 如果缺失值的样本适中，而该属性又是连续型的，有时候就会考虑给定一个step(比如这里的age，我们可以考虑每隔2/3岁为一个步长)，然后把它离散化，之后把NaN作为一个type加到属性类目中。
>4. 有些情况下，缺失的值个数并不是特别多，那我们也可以试着根据已有的值，拟合一下数据，补充上。

在Age值的处理上，这里可以采用拟合补全的方法。(比如用机器学习里面的回归)

In [13]:
from sklearn.ensemble import RandomForestRegressor

# 使用RandomForestRegressor填补缺失的年龄属性
def set_missing_ages(df):
    # 把已有的数值型特征取出丢进RandomForestRegressor中
    age_df = df[['Age','Fare','Parch','SibSp','Pclass']]
    
    known_age = age_df[age_df.Age.notnull()].as_matrix()
    unknown_age = age_df[age_df.Age.isnull()].as_matrix()
    
    y = known_age[:,0]
    
    X = known_age[:,1:]
    
    # fit到RandomForestRegressor中
    rfr = RandomForestRegressor(random_state=0,n_estimators=2000,n_jobs=-1)
    rfr.fit(X,y)
    
    # 用模型进行未知年龄结果预测
    predictedAges = rfr.predict(unknown_age[:,1::])
    
    # 用预测结果补全原缺失数据
    df.loc[(df.Age.isnull()),'Age'] = predictedAges
    
    return df,rfr

def set_Cabin_type(df):
    df.loc[(df.Cabin.notnull()),'Cabin'] = 'Yes'
    df.loc[(df.Cabin.isnull()),'Cabin'] = 'No'
    return df

data_train,rfr = set_missing_ages(data_train)
data_train = set_Cabin_type(data_train)
data_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,No,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,Yes,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,No,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,Yes,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,No,S
5,6,0,3,"Moran, Mr. James",male,23.828953,0,0,330877,8.4583,No,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,Yes,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,No,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,No,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,No,C


训练出来了，现在没有缺失值了，看看:

In [14]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


当然那个Embarked少了两个，可以直接忽略。。

那么特征分析的差不多了，要开始进行学习分类了。但还有一步需要做做。在这些特征中，有些特征是类目型特征并不是数值，而我们通常机器学习是需要用到的是数值型的特征，那么我们就该把对应的类目型特征转换为数值型特征。怎么做呢？

一般来说可以因子化/one-hot编码。

而且pandas里面有个get_dummies函数可以干这个事哦。

In [15]:
# Cabin，原本是一个属性，取值为['Yes','No']，现在因子化后，就变成两个属性’Cabin_yes‘和’Cabin_No‘
# 原本Cabin取值为yes的，在此处的'Cabin_yes'下取值为1，在'Cabin_no'下取值为0；反之亦然
# 其它类目型同理

dummies_Cabin = pd.get_dummies(data_train['Cabin'],prefix='Cabin')

dummies_Embarked = pd.get_dummies(data_train['Embarked'],prefix='Embarked')

dummies_Sex = pd.get_dummies(data_train['Sex'],prefix='Sex')

dummies_Pclass = pd.get_dummies(data_train['Pclass'],prefix='Pclass')

df = pd.concat([data_train,dummies_Cabin,dummies_Embarked,dummies_Sex,dummies_Pclass],axis=1)
# 丢掉一些不需要的值了
df.drop(['Pclass','Name','Sex','Ticket','Cabin','Embarked'],axis=1,inplace=True)
df

,PassengerId,Survived,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,0,22.000000,1,0,7.2500,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,2,1,38.000000,1,0,71.2833,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,3,1,26.000000,0,0,7.9250,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,4,1,35.000000,1,0,53.1000,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
4,5,0,35.000000,0,0,8.0500,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
5,6,0,23.828953,0,0,8.4583,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
6,7,0,54.000000,0,0,51.8625,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
7,8,0,2.000000,3,1,21.0750,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,9,1,27.000000,0,2,11.1333,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
9,10,1,14.000000,1,0,30.0708,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


现在可以看到特征全是数值型的，并且都是我们需要的特征了。

In [16]:
# 再次打印数据描述来看看
df.describe()

,PassengerId,Survived,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,29.647056,0.523008,0.381594,32.204208,0.771044,0.228956,0.188552,0.086420,0.722783,0.352413,0.647587,0.242424,0.206510,0.551066
std,257.353842,0.486592,13.629946,1.102743,0.806057,49.693429,0.420397,0.420397,0.391372,0.281141,0.447876,0.477990,0.477990,0.428790,0.405028,0.497665
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,21.000000,0.000000,0.000000,7.910400,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,29.000000,0.000000,0.000000,14.454200,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,36.000000,1.000000,0.000000,31.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


我们可以看到Age和Fare的范围跨度有点大啊，分别是0.42~80，0~512。这应该要处理一下了，不然会影响到收敛速度的，因此需要对这两个数进行所说的归一化。话说还有PassengerId呢，这属性一看就是个id，不是特征，就不管了。

In [17]:
# 数据分析完，就开始对数据进行预处理了，比如归一化。
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(df['Age'])
df['Age_scaled'] = scaler.fit_transform(df['Age'],age_scale_param)
fare_scale_param = scaler.fit(df['Fare'])
df['Fare_scaled'] = scaler.fit_transform(df['Fare'],fare_scale_param)
df

/Users/wangsenke/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/wangsenke/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/wangsenke/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1,

,PassengerId,Survived,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
0,1,0,22.000000,1,0,7.2500,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-0.561363,-0.502445
1,2,1,38.000000,1,0,71.2833,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.613182,0.786845
2,3,1,26.000000,0,0,7.9250,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.267727,-0.488854
3,4,1,35.000000,1,0,53.1000,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.392955,0.420730
4,5,0,35.000000,0,0,8.0500,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.392955,-0.486337
5,6,0,23.828953,0,0,8.4583,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.427102,-0.478116
6,7,0,54.000000,0,0,51.8625,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.787727,0.395814
7,8,0,2.000000,3,1,21.0750,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-2.029545,-0.224083
8,9,1,27.000000,0,2,11.1333,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.194318,-0.424256
9,10,1,14.000000,1,0,30.0708,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.148636,-0.042956


数据该处理的处理完了，接下来就可以进行相应的建模了。一般就使用sklearn来进行，因为是分类，这里就先选择逻辑回归来看看效果。

使用sklearn库，需要将各特征转换成numpy格式，那么就来进行了：

In [40]:
from sklearn import linear_model

# 把df中的feature字段取出来，转成numpy格式
train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
# train_df
train_np = train_df.as_matrix()
# train_np

# y是Survival的结果
y = train_np[:,0]

# X为特征属性
X = train_np[:,1:]

# fit到逻辑回归中
clf = linear_model.LogisticRegression(C=1.0,penalty='l1',tol=1e-6)
clf.fit(X,y)

clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [41]:
X.shape

(891, 14)

上面只是处理了训练集的数据，还有组测试集的数据还没处理呢。下面就来处理测试集的数据，其实和处理训练集的时候是一样一样的。

In [42]:
data_test = pd.read_csv("test.csv")
data_test.loc[(data_test.Fare.isnull()),'Fare'] = 0
# 对test_data做和train_data中一致的特征变换
# 同样的RandomForestRegressor模型填上丢失的年龄
tmp_df = data_test[['Age','Fare', 'Parch', 'SibSp', 'Pclass']]
null_age = tmp_df[data_test.Age.isnull()].as_matrix()

# 补上年龄
X = null_age[:,1:]
predictedAges = rfr.predict(X)
data_test.loc[(data_test.Age.isnull()),'Age'] = predictedAges

data_test = set_Cabin_type(data_test)
dummies_Cabin = pd.get_dummies(data_test['Cabin'], prefix= 'Cabin')
dummies_Embarked = pd.get_dummies(data_test['Embarked'], prefix= 'Embarked')
dummies_Sex = pd.get_dummies(data_test['Sex'], prefix= 'Sex')
dummies_Pclass = pd.get_dummies(data_test['Pclass'], prefix= 'Pclass')

df_test = pd.concat([data_test, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
df_test.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df_test['Age_scaled'] = scaler.fit_transform(df_test['Age'], age_scale_param)
df_test['Fare_scaled'] = scaler.fit_transform(df_test['Fare'], fare_scale_param)
df_test

/Users/wangsenke/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/wangsenke/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/wangsenke/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1,

,PassengerId,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
0,892,34.500000,0,0,7.8292,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.307535,-0.496637
1,893,47.000000,1,0,7.0000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.256230,-0.511497
2,894,62.000000,0,0,9.6875,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.394665,-0.463335
3,895,27.000000,0,0,8.6625,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-0.261683,-0.481704
4,896,22.000000,1,1,12.2875,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.641161,-0.416740
5,897,14.000000,0,0,9.2250,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.248326,-0.471623
6,898,30.000000,0,0,7.6292,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.033996,-0.500221
7,899,26.000000,1,1,29.0000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-0.337578,-0.117238
8,900,18.000000,0,0,7.2292,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.944743,-0.507390
9,901,21.000000,2,0,24.1500,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-0.717056,-0.204154


In [43]:
test = df_test.filter(regex='Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
predictions = clf.predict(test)

result = pd.DataFrame({'PassengerId':data_test['PassengerId'].as_matrix(),'Survived':predictions.astype(np.int32)})
result.to_csv("logistic_regression_predictions.csv",index=False)


In [44]:
pd.read_csv("logistic_regression_predictions.csv")

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


现在预测出来了，不过还有个问题。上面在实际操作中，我们直接用训练集训练完就将模型用在了测试集上，根本没有过交叉验证好嘛！也没有判断过当前的这个模型是否过拟合啊啥的。

因此可以判断判断当前模型所处的状态啊。
因为如果不断的分析，然后特征还不断的增加，训练的模型在训练集上就会越来越准确，但会逐步失去泛化能力，预测不行，这样就过拟合了呢。

sklearn中有学习曲线，我们可以看看。

In [48]:
import matplotlib.pyplot as plt
from sklearn.learning_curve import learning_curve

# 用sklearn的learning_curve得到training_score和cv_score，使用matplotlib画出learning curve
def plot_learning_curve(estimator,title,X,y,ylim=None,cv=None,n_jobs=1,
                       train_sizes=np.linspace(.05,1.,20),verbose=0,plot=True):
    """
    estimator : 你用的分类器。
    title : 表格的标题。
    X : 输入的feature，numpy类型
    y : 输入的target vector
    ylim : tuple格式的(ymin, ymax), 设定图像中纵坐标的最低点和最高点
    cv : 做cross-validation的时候，数据分成的份数，其中一份作为cv集，其余n-1份作为training(默认为3份)
    n_jobs : 并行的的任务数(默认1)
    """
    train_sizes,train_scores,test_scores = learning_curve(estimator,X,y,cv=cv,n_jobs=n_jobs,train_sizes=train_sizes,verbose=verbose)
    
    train_scores_mean = np.mean(train_scores,axis=1)
    train_scores_std = np.std(train_scores,axis=1)
    test_scores_mean = np.mean(test_scores,axis=1)
    test_scores_std = np.std(test_scores,axis=1)
    
    if plot:
        plt.figure()
        plt.title(title)
        if ylim is not None:
            plt.ylim(*ylim)
        plt.xlabel(u"训练样本数")
        plt.ylabel(u"得分")
        plt.gca().invert_yaxis()
        plt.grid()
        
        plt.fill_between(train_sizes,train_scores_mean - train_scores_std,train_scores_mean + train_scores_std,
                        alpha=0.1,color='b')
        plt.fill_between(train_sizes,test_scores_mean - test_scores_std,test_scores_mean + test_scores_std,
                        alpha=0.1,color='r')
        plt.plot(train_sizes,train_scores_mean,'o-',color='b',label=u"训练集上得分")
        plt.plot(train_sizes,test_scores_mean,'o-',color='r',label=u"交叉验证集上得分")
        
        plt.legend(loc='best')
        
        plt.draw()
        plt.gca().invert_yaxis()
        plt.show()
    
    midpoint = ((train_scores_mean[-1] + train_scores_std[-1]) + (test_scores_mean[-1] - test_scores_std[-1]))/2
    diff = (train_scores_mean[-1] + train_scores_std[-1]) - (test_scores_mean[-1] - test_scores_std[-1])
    return midpoint,diff

# plot_learning_curve(clf,u"学习曲线",X,y)


下面来看看做交叉验证。

In [49]:
pd.DataFrame({"columns":list(train_df.columns)[1:],"coef":list(clf.coef_.T)})

,coef,columns
0,[-0.344228717586],SibSp
1,[-0.104931227735],Parch
2,[0.0],Cabin_No
3,[0.90214087632],Cabin_Yes
4,[0.0],Embarked_C
5,[0.0],Embarked_Q
6,[-0.417262900914],Embarked_S
7,[1.95657802147],Sex_female
8,[-0.677418562801],Sex_male
9,[0.341142634001],Pclass_1


上面模型的系数和最后的结果是一个相关的关系。
这里就可以分析这些相关性。

上面可以看到：有Cabin的会大程度的提高获救概率；Sex是female的极大提升最后获救的概率，而是male的则拉低这个概率；Pclass_1的能提升，Pclass_3会极大拉低概率。
Age是个负相关，说明在模型中，年龄越小，越容易获救。

有了初步想法后，怎么样知道哪些优化方法是有希望的呢？靠交叉验证咯。


In [63]:
from sklearn import cross_validation

# 简单看看打分情况
clf = linear_model.LogisticRegression(C=1.0,penalty='l1',tol=1e-6)
all_data = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
X = all_data.as_matrix()[:,1:]
y = all_data.as_matrix()[:,0]
print cross_validation.cross_val_score(clf,X,y,cv=5)

# plot_learning_curve(clf,u"学习曲线",X,y)

# 分割数据
split_train,split_cv = cross_validation.train_test_split(df,test_size=0.3,random_state=0)
train_df = split_train.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
# 生成模型
clf = linear_model.LogisticRegression(C=1.0,penalty='l1',tol=1e-6)
clf.fit(train_df.as_matrix()[:,1:],train_df.as_matrix()[:,0])

# 对交叉验证集进行预测
cv_df = split_cv.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
predictions = clf.predict(cv_df.as_matrix()[:,1:])
# cv_df.as_matrix()[:,0].shape
split_cv[predictions != cv_df.as_matrix()[:,0]]

[ 0.81564246  0.81564246  0.78651685  0.78651685  0.81355932]


,PassengerId,Survived,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
298,299,1,41.574877,0,0,30.5000,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.875610,-0.034314
14,15,0,14.000000,0,0,7.8542,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-1.148636,-0.490280
803,804,1,0.420000,0,1,8.5167,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-2.145531,-0.476941
474,475,0,22.000000,0,0,9.8375,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-0.561363,-0.450347
642,643,0,2.000000,3,2,27.9000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-2.029545,-0.086664
55,56,1,44.064830,0,0,35.5000,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.058395,0.066360
587,588,1,60.000000,1,1,79.2000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.228181,0.946246
740,741,1,38.426632,0,0,30.0000,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.644500,-0.044381
839,840,1,50.910950,0,0,29.7000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.560963,-0.050421
301,302,1,23.312621,2,0,23.2500,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.465005,-0.180290


In [65]:
# 去除预测错误的
origin_data_train = pd.read_csv("Train.csv")
bad_cases = origin_data_train.loc[origin_data_train['PassengerId'].isin(split_cv[predictions!=cv_df.as_matrix()[:,0]]['PassengerId'].values)]
bad_cases

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.00,0,0,350406,7.8542,NaN,S
49,50,0,3,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,18.00,1,0,349237,17.8000,NaN,S
55,56,1,1,"Woolner, Mr. Hugh",male,NaN,0,0,19947,35.5000,C52,S
65,66,1,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,NaN,C
68,69,1,3,"Andersson, Miss. Erna Alexandra",female,17.00,4,2,3101281,7.9250,NaN,S
85,86,1,3,"Backstrom, Mrs. Karl Alfred (Maria Mathilda Gu...",female,33.00,3,0,3101278,15.8500,NaN,S
113,114,0,3,"Jussila, Miss. Katriina",female,20.00,1,0,4136,9.8250,NaN,S
140,141,0,3,"Boulos, Mrs. Joseph (Sultana)",female,NaN,0,2,2678,15.2458,NaN,C
204,205,1,3,"Cohen, Mr. Gurshon ""Gus""",male,18.00,0,0,A/5 3540,8.0500,NaN,S
240,241,0,3,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C
